In [ ]:
import pandas as pd
import re
import itertools
from collections import defaultdict

class ApprOXimate:
    def __init__(self, csv_file_path):
        self.load_chemical_data(csv_file_path)
    
    def load_chemical_data(self, csv_file_path):
        df = pd.read_csv(csv_file_path)
        self.list1 = defaultdict(list)
        self.list2 = {}
        
        for _, row in df.iterrows():
            element = row['Element']
            ox = int(row['Ox'])
            red = int(row['Red'])
            srp = float(row['SRP (V)'])
            self.list1[element].append((ox, red, srp))
        
        # Fixed oxidation states for common elements
        fixed_oxidation_states = {
            'O': -2, 'Na': 1
        }
        
        for element, ox in fixed_oxidation_states.items():
            self.list2[element] = ox
    
    def parse_formula(self, formula_input):
        pattern = r'([A-Z][a-z]*)(\d*\.?\d*/?\d*\.?\d*)'
        matches = re.findall(pattern, formula_input)
        formula = {}
        
        for element, quantity_str in matches:
            if element not in self.list1 and element not in self.list2:
                print(f'Warning: Element {element} not found in database')
                return None
            
            quantity = 1.0
            if quantity_str:
                if '/' in quantity_str:
                    numerator, denominator = quantity_str.split('/')
                    quantity = float(numerator) / float(denominator)
                else:
                    quantity = float(quantity_str)
            
            formula[element] = formula.get(element, 0) + quantity
        
        return formula

    def charge_balance(self, formula):
        local_list1 = defaultdict(list)
        for element, states in self.list1.items():
            local_list1[element] = states.copy()
        
        total_charge = 0
        previous_charge_with_delta = float('nan')
        element_with_final_lowest_srp = None
        final_delta_ox_max = 0
        final_charge_with_delta = 0
        element_quantity = 0
        final_ox = 0
        final_red = 0
        
        iteration_count = 0
        max_iterations = 100
        
        while True:
            iteration_count += 1
            if iteration_count > max_iterations:
                break
            
            sum_non_m = 0
            sum_m = 0
            
            for element, quantity in formula.items():
                if element in self.list2:
                    red = self.list2[element]
                    sum_non_m += quantity * red
                else:
                    data = next((x for x in local_list1[element] if x[1] > 0), None)
                    if data:
                        sum_m += quantity * data[1]
            
            total_charge = round(sum_non_m + sum_m, 5)
            
            # Find element with lowest SRP
            lowest_srp_element = None
            lowest_srp = float('inf')
            
            for element, quantity in formula.items():
                if element not in self.list2:
                    data = next((x for x in local_list1[element] if x[1] > 0), None)
                    if data and data[2] < lowest_srp:
                        lowest_srp = data[2]
                        lowest_srp_element = {
                            'element': element,
                            'quantity': quantity,
                            'data': data
                        }
            
            if not lowest_srp_element:
                break
            
            delta_ox_max = round((lowest_srp_element['data'][0] - lowest_srp_element['data'][1]) * lowest_srp_element['quantity'], 5)
            charge_with_delta = round(delta_ox_max + total_charge, 5)
            
            if charge_with_delta == previous_charge_with_delta:
                break
            
            previous_charge_with_delta = charge_with_delta
            
            if charge_with_delta < 0:
                total_charge += delta_ox_max
                updated_data = next((x for x in local_list1[lowest_srp_element['element']] 
                                  if x[1] == lowest_srp_element['data'][1]), None)
                if updated_data:
                    new_data = next((x for x in local_list1[lowest_srp_element['element']] 
                                   if x[1] == updated_data[0]), None)
                    if not new_data:
                        new_data = (updated_data[0], updated_data[0], float('inf'))
                    
                    local_list1[lowest_srp_element['element']] = [
                        x for x in local_list1[lowest_srp_element['element']] 
                        if x[1] != updated_data[1]
                    ]
                    local_list1[lowest_srp_element['element']].append(new_data)
            else:
                element_with_final_lowest_srp = lowest_srp_element['element']
                final_delta_ox_max = delta_ox_max
                final_charge_with_delta = charge_with_delta
                element_quantity = lowest_srp_element['quantity']
                final_ox = lowest_srp_element['data'][0]
                final_red = lowest_srp_element['data'][1]
                break
        
        # Calculate final charge
        final_charge = 0
        for element, quantity in formula.items():
            if element in self.list2:
                final_charge += quantity * self.list2[element]
            elif element != element_with_final_lowest_srp:
                final_data = next((x for x in local_list1[element] if x[1] > 0), None)
                if final_data:
                    final_charge += quantity * final_data[1]
        
        if element_with_final_lowest_srp and (final_charge_with_delta == 0 or final_charge_with_delta > 0):
            if final_delta_ox_max != 0:  # Only adjust if there's actually a difference in oxidation states
                adjusted_quant = round(element_quantity * (final_charge_with_delta / final_delta_ox_max), 5)
                remaining_quant = round(element_quantity - adjusted_quant, 5)
                
                if adjusted_quant > 0:
                    final_charge += adjusted_quant * final_red
                if remaining_quant > 0:
                    final_charge += remaining_quant * final_ox
            else:
                # If no difference in oxidation states, just use the original quantity
                final_charge += element_quantity * final_red
        
        final_charge = round(final_charge, 5)
        
        # Build result string
        result_parts = []
        
        # Add fixed elements first
        for element in sorted(formula.keys()):
            if element in self.list2:
                result_parts.append(f"{element}:{self.list2[element]}:{formula[element]}")
        
        # Add variable elements
        for element in sorted(formula.keys()):
            if element not in self.list2 and element != element_with_final_lowest_srp:
                final_data = next((x for x in local_list1[element] if x[1] > 0), None)
                if final_data:
                    result_parts.append(f"{element}:{final_data[1]}:{formula[element]}")
        
        # Handle element with lowest SRP
        if element_with_final_lowest_srp and (final_charge_with_delta == 0 or final_charge_with_delta > 0):
            if final_delta_ox_max != 0:
                adjusted_quant = round(element_quantity * (final_charge_with_delta / final_delta_ox_max), 5)
                remaining_quant = round(element_quantity - adjusted_quant, 5)
                
                if adjusted_quant > 0:
                    result_parts.append(f"{element_with_final_lowest_srp}:{final_red}:{adjusted_quant}")
                if remaining_quant > 0:
                    result_parts.append(f"{element_with_final_lowest_srp}:{final_ox}:{remaining_quant}")
            else:
                # If no difference in oxidation states, just show the original quantity
                result_parts.append(f"{element_with_final_lowest_srp}:{final_red}:{element_quantity}")
        
        result_parts.append(f"FinalChargeBalance:{final_charge}")
        
        return ";".join(result_parts)
    
true_outputs = pd.read_csv("Correct Outputs.csv")
test_cases = true_outputs['Chemical'].unique().tolist()

# Create analyzer
analyzer = ApprOXimate('all_elements_oxidation_states(in).csv')

for formula in test_cases:
    parsed = analyzer.parse_formula(formula)
    if parsed:
        result = analyzer.charge_balance(parsed)
        correct_row = true_outputs[true_outputs['Chemical'] == formula]
        
        result_parts = result.split(";")
        result_dict = {part.split(":")[0]: part for part in result_parts}
        correct_dict = {part.split(":")[0]: part for part in correct_row['Correct Output'].values[0].split(";")}
        
        final_charge_balance_result = float(result_dict['FinalChargeBalance'].split(':')[1])
        final_correct_charge_balance = float(correct_dict['FinalChargeBalance'].split(':')[1])
        
        total_difference = abs(final_charge_balance_result - final_correct_charge_balance)
        
        if total_difference > 0.01:
            print(f"Discrepancy found for {formula}: {total_difference}, {result}")

Discrepancy found for Na0.5Mn1/9Cu8/27Fe8/27Ni8/27O2: 0.441, Na:1:0.5;O:-2:2.0;Cu:2:0.2962962962962963;Fe:3:0.2962962962962963;Mn:3:0.1111111111111111;Ni:4:0.2962962962962963;FinalChargeBalance:-0.5


In [6]:
import ipywidgets as widgets
from IPython.display import display

# Create the text input widget
text_input = widgets.Text(
    value='',
    description='Type here:',
    disabled=False
)

# Create an HTML widget to show the status/output
status_output = widgets.HTML(value="The text will appear here after you type something. :)")

# Define the function to handle changes
def on_text_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        status_output.value = f"You entered: {change['new']}"

# Observe changes in the text input
text_input.observe(on_text_change, names='value')

# Display the widgets
display(text_input, status_output)

Text(value='', description='Type here:')

HTML(value='The text will appear here after you type something. :)')

In [7]:
analyzer = ApprOXimate('all_elements_oxidation_states(in).csv')
# parsed = analyzer.parse_formula(formula)
# result = analyzer.charge_balance(parsed)

import ipywidgets as widgets
from IPython.display import display

# Sample function that processes the input
def process_text(input_text):
    text_output = ""
    
    parsed = analyzer.parse_formula(input_text)
    result = analyzer.charge_balance(parsed)
    result_parts = result.split(";")
    for item in result_parts:
        if item.startswith("FinalChargeBalance:"):
            final_charge, value = item.split(":")
            result_parts = [f"Final Charge Balance is {value}"] + result_parts[1:]
            text_output += f"Final Charge Balance is {value}"
        else:
            element, oxstate, quantity = item.split(":")
            result_parts = [f"Element: {element} Quantity: {quantity} Oxidation State: {oxstate}"] + result_parts[1:]  # Keep the first part intact
            text_output += f"Element: {element} Quantity: {quantity} Oxidation State: {oxstate}<br>"
            
        result = text_output
    return result

# Widgets
text_input = widgets.Text(
    value='',
    description='Type here:',
    disabled=False
)

button = widgets.Button(
    description='Calculate Oxidation States',
    button_style='info',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to process text',
    icon='check',
    layout=widgets.Layout(width='250px', height='40px')
)

output_display = widgets.HTML(value="Result will appear here")

# Button click handler
def on_button_click(b):
    user_input = text_input.value
    result = process_text(user_input)
    output_display.value = result

# Attach handler to button
button.on_click(on_button_click)

# Layout
ui = widgets.VBox([text_input, button, output_display])
display(ui)


In [10]:
import pandas as pd
import re
import itertools
from collections import defaultdict

class ApprOXimate:
    def __init__(self, csv_file_path):
        self.load_chemical_data(csv_file_path)
    
    def load_chemical_data(self, csv_file_path):
        df = pd.read_csv(csv_file_path)
        self.list1 = defaultdict(list)
        self.list2 = {}
        
        for _, row in df.iterrows():
            element = row['Element']
            ox = int(row['Ox'])
            red = int(row['Red'])
            srp = float(row['SRP (V)'])
            self.list1[element].append((ox, red, srp))
        
        # Fixed oxidation states for common elements
        fixed_oxidation_states = {
            'O': -2, 'Na': 1
        }
        
        for element, ox in fixed_oxidation_states.items():
            self.list2[element] = ox
    
    def parse_formula(self, formula_input):
        pattern = r'([A-Z][a-z]*)(\d*\.?\d*/?\d*\.?\d*)'
        matches = re.findall(pattern, formula_input)
        formula = {}
        
        for element, quantity_str in matches:
            if element not in self.list1 and element not in self.list2:
                print(f'Warning: Element {element} not found in database')
                return None
            
            quantity = 1.0
            if quantity_str:
                if '/' in quantity_str:
                    numerator, denominator = quantity_str.split('/')
                    quantity = float(numerator) / float(denominator)
                else:
                    quantity = float(quantity_str)
            
            formula[element] = formula.get(element, 0) + quantity
        
        return formula

    def charge_balance(self, formula):
        local_list1 = defaultdict(list)
        for element, states in self.list1.items():
            local_list1[element] = states.copy()
        
        total_charge = 0
        previous_charge_with_delta = float('nan')
        element_with_final_lowest_srp = None
        final_delta_ox_max = 0
        final_charge_with_delta = 0
        element_quantity = 0
        final_ox = 0
        final_red = 0
        
        iteration_count = 0
        max_iterations = 100
        
        while True:
            iteration_count += 1
            if iteration_count > max_iterations:
                break
            
            sum_non_m = 0
            sum_m = 0
            
            for element, quantity in formula.items():
                if element in self.list2:
                    red = self.list2[element]
                    sum_non_m += quantity * red
                else:
                    data = next((x for x in sorted(local_list1[element], key=lambda y: -y[1]) if x[1] > 0), None)
                    if data:
                        sum_m += quantity * data[1]
            
            total_charge = round(sum_non_m + sum_m, 5)
            
            # Find element with lowest SRP
            lowest_srp_element = None
            lowest_srp = float('inf')
            
            for element, quantity in formula.items():
                if element not in self.list2:
                    data = next((x for x in sorted(local_list1[element], key=lambda y: -y[1]) if x[1] > 0), None)
                    if data and data[2] < lowest_srp:
                        lowest_srp = data[2]
                        lowest_srp_element = {
                            'element': element,
                            'quantity': quantity,
                            'data': data
                        }
            
            if not lowest_srp_element:
                break
            
            delta_ox_max = round((lowest_srp_element['data'][0] - lowest_srp_element['data'][1]) * lowest_srp_element['quantity'], 5)
            charge_with_delta = round(delta_ox_max + total_charge, 5)
            
            if charge_with_delta == previous_charge_with_delta:
                break
            
            previous_charge_with_delta = charge_with_delta
            
            if charge_with_delta < 0:
                total_charge += delta_ox_max
                updated_data = next((x for x in local_list1[lowest_srp_element['element']] 
                                  if x[1] == lowest_srp_element['data'][1]), None)
                if updated_data:
                    new_data = next((x for x in local_list1[lowest_srp_element['element']] 
                                   if x[1] == updated_data[0]), None)
                    if not new_data:
                        new_data = (updated_data[0], updated_data[0], float('inf'))
                    
                    local_list1[lowest_srp_element['element']] = [
                        x for x in local_list1[lowest_srp_element['element']] 
                        if x[1] != updated_data[1]
                    ]
                    local_list1[lowest_srp_element['element']].append(new_data)
            else:
                element_with_final_lowest_srp = lowest_srp_element['element']
                final_delta_ox_max = delta_ox_max
                final_charge_with_delta = charge_with_delta
                element_quantity = lowest_srp_element['quantity']
                final_ox = lowest_srp_element['data'][0]
                final_red = lowest_srp_element['data'][1]
                break
        
        # Calculate final charge
        final_charge = 0
        for element, quantity in formula.items():
            if element in self.list2:
                final_charge += quantity * self.list2[element]
            elif element != element_with_final_lowest_srp:
                final_data = next((x for x in local_list1[element] if x[1] > 0), None)
                if final_data:
                    final_charge += quantity * final_data[1]
        
        if element_with_final_lowest_srp and (final_charge_with_delta == 0 or final_charge_with_delta > 0):
            if final_delta_ox_max != 0:  # Only adjust if there's actually a difference in oxidation states
                adjusted_quant = round(element_quantity * (final_charge_with_delta / final_delta_ox_max), 5)
                remaining_quant = round(element_quantity - adjusted_quant, 5)
                
                if adjusted_quant > 0:
                    final_charge += adjusted_quant * final_red
                if remaining_quant > 0:
                    final_charge += remaining_quant * final_ox
            else:
                # If no difference in oxidation states, just use the original quantity
                final_charge += element_quantity * final_red
        
        final_charge = round(final_charge, 5)
        
        # Build result string
        result_parts = []
        
        # Add fixed elements first
        for element in sorted(formula.keys()):
            if element in self.list2:
                result_parts.append(f"{element}:{self.list2[element]}:{formula[element]}")
        
        # Add variable elements
        for element in sorted(formula.keys()):
            if element not in self.list2 and element != element_with_final_lowest_srp:
                final_data = next((x for x in local_list1[element] if x[1] > 0), None)
                if final_data:
                    result_parts.append(f"{element}:{final_data[1]}:{formula[element]}")
        
        # Handle element with lowest SRP
        if element_with_final_lowest_srp and (final_charge_with_delta == 0 or final_charge_with_delta > 0):
            if final_delta_ox_max != 0:
                adjusted_quant = round(element_quantity * (final_charge_with_delta / final_delta_ox_max), 5)
                remaining_quant = round(element_quantity - adjusted_quant, 5)
                
                if adjusted_quant > 0:
                    result_parts.append(f"{element_with_final_lowest_srp}:{final_red}:{adjusted_quant}")
                if remaining_quant > 0:
                    result_parts.append(f"{element_with_final_lowest_srp}:{final_ox}:{remaining_quant}")
            else:
                # If no difference in oxidation states, just show the original quantity
                result_parts.append(f"{element_with_final_lowest_srp}:{final_red}:{element_quantity}")
        
        result_parts.append(f"FinalChargeBalance:{final_charge}")
        
        return ";".join(result_parts)
    
true_outputs = pd.read_csv("Correct Outputs.csv")
test_cases = true_outputs['Chemical'].unique().tolist()

# Create analyzer
analyzer = ApprOXimate('all_elements_oxidation_states(in).csv')

for formula in test_cases:
    parsed = analyzer.parse_formula(formula)
    if parsed:
        result = analyzer.charge_balance(parsed)
        correct_row = true_outputs[true_outputs['Chemical'] == formula]
        
        result_parts = result.split(";")
        result_dict = {part.split(":")[0]: part for part in result_parts}
        correct_dict = {part.split(":")[0]: part for part in correct_row['Correct Output'].values[0].split(";")}
        
        final_charge_balance_result = float(result_dict['FinalChargeBalance'].split(':')[1])
        final_correct_charge_balance = float(correct_dict['FinalChargeBalance'].split(':')[1])
        
        total_difference = abs(final_charge_balance_result - final_correct_charge_balance)
        
        if total_difference > 0.01:
            print(f"Discrepancy found for {formula}: {total_difference}, {result}")

Discrepancy found for NaMnO2: 2.33332, Na:1:1.0;O:-2:2.0;Mn:4:1.33333;FinalChargeBalance:2.33332
Discrepancy found for NaCu0.5Fe0.5O2: 1.0, Na:1:1.0;O:-2:2.0;Cu:2:0.5;Fe:1:0.5;FinalChargeBalance:-1.5
Discrepancy found for NaCu0.5Mn0.5O2: 0.5, Na:1:1.0;O:-2:2.0;Mn:3:0.5;Cu:2:0.5;FinalChargeBalance:-0.5
Discrepancy found for NaFe0.5Mn0.5O2: 0.5, Na:1:1.0;O:-2:2.0;Mn:3:0.5;Fe:2:0.5;FinalChargeBalance:-0.5
Discrepancy found for NaFe0.5Ni0.5O2: 0.5, Na:1:1.0;O:-2:2.0;Fe:1:0.5;Ni:4:0.5;FinalChargeBalance:-0.5
Discrepancy found for NaMn0.5Ni0.5O2: 0.5, Na:1:1.0;O:-2:2.0;Mn:3:0.5;Ni:2:0.5;FinalChargeBalance:-0.5
Discrepancy found for NaCu1/3Fe1/3Mn1/3O2: 0.33334, Na:1:1.0;O:-2:2.0;Cu:2:0.3333333333333333;Mn:3:0.3333333333333333;Fe:3:0.33333;FinalChargeBalance:-0.33334
Discrepancy found for NaCu1/3Mn1/3Ni1/3O2: 0.33335, Na:1:1.0;O:-2:2.0;Cu:1:0.3333333333333333;Mn:3:0.3333333333333333;Ni:4:0.33333;FinalChargeBalance:-0.33335
Discrepancy found for NaFe1/3Mn1/3Ni1/3O2: 0.66669, Na:1:1.0;O:-2:2.0;